# Phase 2: Checkpoint Definition (Outline Creation)

This notebook handles:
1. Loading results from Phase 1
2. Generating draft request checkpoint titles and goals
3. Creating a structured outline of major checkpoints
4. Saving results for Phase 3

In [1]:
# Import shared utilities
import sys
sys.path.append('.')

In [2]:
from google.genai import types
from mirascope.core import google
from mirascope import Messages
from pydantic import BaseModel, Field
from typing import List, Optional

from utils.interim_data_management import load_interim_data, save_interim_data
from utils.config import get_config

## Load Phase 1 Results

In [3]:
# Get config
config = get_config()

# Load data from Phase 1
raw_inputs = load_interim_data(config.get("INTERIM_DATA_DIR"), "raw_inputs", "phase1")
files_dict = load_interim_data(config.get("INTERIM_DATA_DIR"), "files_dict", "phase1")
file_analyses = load_interim_data(config.get("INTERIM_DATA_DIR"), "file_analyses", "phase1")
phase1_summary = load_interim_data(config.get("INTERIM_DATA_DIR"), "phase1_summary", "phase1")

# Extract key variables
user_instructions = raw_inputs["user_instructions"]
file_tree = raw_inputs["file_tree"]
GLOBAL_CODEBASE_PURPOSE = raw_inputs["global_codebase_purpose"]

print(f"Loaded Phase 1 data:")
print(f"- Files analyzed: {len(file_analyses)}")
print(f"- User instructions length: {len(user_instructions)} characters")

Loaded raw_inputs from interim_data/phase1/raw_inputs.json
Loaded files_dict from interim_data/phase1/files_dict.json
Loaded file_analyses from interim_data/phase1/file_analyses.json
Loaded phase1_summary from interim_data/phase1/phase1_summary.json
Loaded Phase 1 data:
- Files analyzed: 55
- User instructions length: 8679 characters


In [4]:
# Reconstruct file analysis objects
from pydantic import BaseModel

class InitialFileAnalysisItemWithContent(BaseModel):
    file_path: str
    overall_relevance_to_request_objective: str
    initial_content_summary_purpose: str
    file_contents: str

# Convert loaded JSON back to Pydantic models
all_analyses_with_content = [
    InitialFileAnalysisItemWithContent(**item) for item in file_analyses
]

## Step 2.1: Define Models and Prompts for Checkpoint Outline

In [5]:
class RequestCheckpointOutline(BaseModel):
    order: int
    title: str
    goal_for_checkpoint: str
    prerequisites: Optional[List[str]] = Field(default_factory=list)
    expected_outcome: Optional[str] = None

class FullRequestCheckpointOutline(BaseModel):
    request_checkpoints: List[RequestCheckpointOutline]

In [6]:
OUTLINE_GENERATION_SYSTEM_PROMPT = """<system_context>
You are an expert request planner and software architect. 
Your task is to break down a confirmed high-level request objective into a sequence of major, ordered "Request Checkpoints." 
These checkpoints represent significant, logical milestones that must be completed sequentially to achieve the overall request goal. 
Each checkpoint is a substantial piece of work that, once completed, moves the request significantly closer to its final state. 
Think of these as the main chapters in the story of this feature's development.
</system_context>

<input_data>
    <confirmed_high_level_request_objective>
    {confirmed_high_level_request_objective_placeholder}
    </confirmed_high_level_request_objective>

    <overall_codebase_purpose>
    {overall_codebase_purpose_placeholder}
    </overall_codebase_purpose>

    <file_tree>
    {file_tree_placeholder}
    </file_tree>

    <initial_file_analysis_summary>
    {initial_file_analysis_summary_placeholder}
    </initial_file_analysis_summary>
</input_data>

<output_instructions>
Based on the <confirmed_high_level_request_objective> and the provided context (<overall_codebase_purpose>, <file_tree>, <initial_file_analysis_summary>), define a sequence of Request Checkpoints.

For each Request Checkpoint, provide the following:
1.  **Order:** A sequential number (e.g., 1, 2, 3, ...).
2.  **Title:** A concise, descriptive title for the checkpoint (e.g., "Setup Core Database Schema for Feature X," "Implement Backend API for Feature X," "Develop Frontend UI Components for Feature X," "Integrate and Test Feature X End-to-End").
3.  **Goal for Checkpoint:** A clear statement (1-2 sentences) detailing what this specific checkpoint aims to achieve and what state the relevant parts of the system should be in upon its completion.
4.  **Prerequisites:** List the `Order` number(s) of any other checkpoint(s) that *must* be completed before this one can begin. For the first checkpoint, this will typically be "None" or "Request Start." Subsequent checkpoints will usually depend on the immediately preceding one.
5.  **Expected Outcome / Deliverables:** Briefly describe the tangible results or key deliverables of completing this checkpoint (e.g., "Migrated database schema," "Functional API endpoints documented in Postman," "Completed UI mockups and core components").

**Key Considerations for Defining Checkpoints:**
*   **Sequential Logic:** The checkpoints MUST represent a logical, ordered progression of work. The output of one checkpoint often forms the input or foundation for the next.
*   **Significant Milestones:** Each checkpoint should represent a meaningful step towards the overall objective. Avoid making them too granular (like individual low-level tasks) or too broad (like the entire request itself). Aim for 3-7 checkpoints for moderately complex features.
*   **Bridging the Gap:** These checkpoints are the primary mechanism for breaking down the high-level objective into manageable, verifiable stages. They define the "journey" to completion.
*   **Cohesive Work Packages:** A checkpoint should group related activities. For example, all backend API work for a specific sub-feature might be one checkpoint.

Please output a list of these checkpoint definitions.
</output_instructions>
"""

OUTLINE_GENERATION_USER_PROMPT = """<user_input>
    <input_text>Please now generation a checkpoint outline to satisfy the following request.</input_text>
    <request>
        {high_level_objective}
    </request>
</user_input>"""

## Step 2.2: Generate Request Checkpoint Outline

In [7]:
def format_request_outline_system_prompt(
    confirmed_high_level_objective: str,
    overall_codebase_purpose: str,
    file_tree: str,
    initial_file_analysis_summary: str
) -> str:
    system_prompt = OUTLINE_GENERATION_SYSTEM_PROMPT.replace(
        "{confirmed_high_level_request_objective_placeholder}", confirmed_high_level_objective.strip()
    ).replace(
        "{overall_codebase_purpose_placeholder}", overall_codebase_purpose.strip()
    ).replace(
        "{file_tree_placeholder}", file_tree
    ).replace(
        "{initial_file_analysis_summary_placeholder}", initial_file_analysis_summary.strip()
    )
    return system_prompt

def format_request_outline_user_prompt(high_level_objective: str) -> str:
    prompt = OUTLINE_GENERATION_USER_PROMPT.replace(
        "{high_level_objective}", high_level_objective.strip()
    )
    return prompt

In [8]:
@google.call(
    "gemini-2.5-pro-preview-05-06",
    response_model=FullRequestCheckpointOutline,
    call_params={
        "config": types.GenerateContentConfig(
            # thinking_config=types.ThinkingConfig(thinking_budget=8192), 
            temperature=0.5
        )
    },
)
def generate_request_outline(system_prompt: str, user_prompt: str) -> str:
    return [Messages.System(system_prompt), Messages.User(user_prompt)]

In [9]:
# Format prompts
checkpoint_outline_system_prompt = format_request_outline_system_prompt(
    user_instructions,  # Using user instructions as the objective for now
    GLOBAL_CODEBASE_PURPOSE, 
    file_tree, 
    '\n'.join([analysis.model_dump_json(indent=2) for analysis in all_analyses_with_content])
)

checkpoint_outline_user_prompt = format_request_outline_user_prompt(
    high_level_objective=user_instructions
)

# Generate the outline
print("Generating request checkpoint outline...")
request_checkpoint_outline = generate_request_outline(
    checkpoint_outline_system_prompt, 
    checkpoint_outline_user_prompt
)

print(f"\nGenerated {len(request_checkpoint_outline.request_checkpoints)} checkpoints")

Generating request checkpoint outline...


/Users/Ford/anaconda3/envs/promo/lib/python3.12/site-packages/google/genai/_common.py:240: UserWarning: null is not a valid Type
  warnings.warn(f"{value} is not a valid {cls.__name__}")



Generated 7 checkpoints


In [10]:
# Display the outline
print("Request Checkpoint Outline:")
print("=" * 50)
for checkpoint in request_checkpoint_outline.request_checkpoints:
    print(f"\n{checkpoint.order}. {checkpoint.title}")
    print(f"   Goal: {checkpoint.goal_for_checkpoint}")
    print(f"   Prerequisites: {checkpoint.prerequisites}")
    print(f"   Expected Outcome: {checkpoint.expected_outcome}")

Request Checkpoint Outline:

1. Implement New Server Actions for Context Bundles
   Goal: Implement the new server actions `resolveAndCreateBundle.ts` and `getContextBundle.ts` under `app/(auth)/context/_actions/context-bundles/` to handle context bundle creation, resolution, and retrieval. This establishes the core backend logic for managing context bundles.
   Prerequisites: ['Request Start']
   Expected Outcome: Functional `resolveAndCreateBundle` and `getContextBundle` server actions. `resolveAndCreateBundle` will correctly process context selections, store a snapshot in the `contextBundles` table, and return a bundle ID. `getContextBundle` will retrieve a specific bundle by ID for the authenticated user.

2. Refactor Legacy `resolveContextSelectionAction`
   Goal: Update the existing `resolveContextSelectionAction` to delegate its functionality to the new `resolveAndCreateBundle` action. This ensures that all existing parts of the application that use the old action will now use t

## Save Phase 2 Results

In [11]:
# Save the checkpoint outline
save_interim_data(config.get("INTERIM_DATA_DIR"), request_checkpoint_outline, "checkpoint_outline", "phase2")

# Save a summary
phase2_summary = {
    "checkpoint_count": len(request_checkpoint_outline.request_checkpoints),
    "checkpoint_titles": [cp.title for cp in request_checkpoint_outline.request_checkpoints],
    "user_instructions_used": user_instructions[:200] + "..."  # First 200 chars
}

save_interim_data(config.get("INTERIM_DATA_DIR"), phase2_summary, "phase2_summary", "phase2")

print("\nPhase 2 completed successfully!")
print(f"Results saved to: {config.get('INTERIM_DATA_DIR')}/phase2/")

Saved checkpoint_outline to interim_data/phase2/checkpoint_outline.json
Saved phase2_summary to interim_data/phase2/phase2_summary.json

Phase 2 completed successfully!
Results saved to: interim_data/phase2/
